# Encontrando insights de la UEFA Champions League

df1 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/Examen/uefa_2020.csv")

df2 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/Examen/uefa_2021.csv")

df3 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/Examen/uefa_2022.csv")

La Liga de Campeones de la UEFA, a menudo llamada Liga de Campeones, es una competencia anual de fútbol por excelencia que cautiva a los fanáticos de todo el mundo. Establecida en 1955 como la Copa de Clubes Campeones de Europa, evolucionó a la Liga de Campeones de la UEFA en 1992, ampliando su atractivo. El formato moderno cuenta con 32 equipos de clubes de primer nivel seleccionados en función de su desempeño en la liga nacional, lo que aumenta la intriga.  

Este evento electrizante trasciende los deportes y se convierte en una celebración de la unidad, la cultura y el orgullo nacional. Los fanáticos, vestidos con los colores de sus países, crean una atmósfera eléctrica, lo que hace que el torneo sea tanto sobre los espectadores como sobre los jugadores. Financieramente, la Liga de Campeones es un salvavidas para los clubes, ya que aumenta los ingresos y ofrece oportunidades transformadoras. Sin embargo, genera debates sobre las disparidades de riqueza en el fútbol europeo.  

La Liga de Campeones es sinónimo de rivalidades históricas, triunfos de los desvalidos y brillantez individual. Para los jugadores, representa la cima de su carrera, mientras que para los fanáticos, es un fenómeno cultural. El himno y los rituales icónicos enriquecen la experiencia futbolística. En 200 palabras, la UEFA Champions League es el epítome de la excelencia del fútbol europeo, que ofrece momentos inolvidables, recompensas económicas y un impacto cultural único, con 32 clubes de primer nivel que se suman a su atractivo.

**Nombres de las tablas o ficheros a utilizar**:  
[uefa_2020.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/EQuW8d16tZJKpYZN6jR480sBgc1IYypAw9hGHKQARS560g?e=7TybBU)  
[uefa_2021.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/ESG8YBrjD3tBky3aGT0MwIABNt_PQxFVVYVlWMnZYUmV8g?e=VrYHtX)  
[uefa_2022.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/Ecf8O8U8sxJLoW9Bkd2ZVzgBb_gZc35mNcprnR9FMlzliQ?e=Vn3a6v)  
- Todas las tablas tienen las mismas columnas y data types

| Column | Definition | Data type |
|--------|------------|-----------|
| `STAGE`| Stage of the March | `VARCHAR(50)` |
| `DATE` | When the match occurred. | `DATE` |
| `PENS` | Did the match end with penalty | `VARCHAR(50)` |
| `PENS_HOME_SCORE` | In case of penalty, score by home team | `VARCHAR(50)` |
| `PENS_AWAY_SCORE` | In case of penalty, score by away team | `VARCHAR(50)` |
| `TEAM_NAME_HOME` | Team home name | `VARCHAR(50)` |
| `TEAM_NAME_AWAY`| Team away  name | `VARCHAR(50)` |
| `TEAM_HOME_SCORE` | Team home score | `NUMBER` |
| `TEAM_AWAY_SCORE` | Team away score | `NUMBER` |
| `POSSESSION_HOME` | Ball possession for the home team | `FLOAT` |
| `POSSESSION_AWAY` | Ball possession for the away team | `FLOAT` |
| `TOTAL_SHOTS_HOME` | Number of shots by the home team | `NUMBER` |
| `TOTAL_SHOTS_AWAY` | Number of shots by the away team | `NUMBER`
| `SHOTS_ON_TARGET_HOME` | Total shot for home team | `FLOAT` |
| `SHOTS_ON_TARGET_AWAY` | Total shot for away team | `FLOAT` |
| `DUELS_WON_HOME` | duel win possession of ball - for home team | `NUMBER` |
| `DUELS_WON_AWAY` | duel win possession of ball - for away team | `NUMBER` 
| `PREDICTION_TEAM_HOME_WIN` | Probability of home team to win | `FLOAT` |
| `PREDICTION_DRAW` | Probability of draw | `FLOAT` |
| `PREDICTION_TEAM_AWAY_WIN` | Probability of away team to win | `FLOAT` |
| `LOCATION` | Stadium where the match was held | `VARCHAR(50)` |

### Requerimientos:  
-Puedes utilizar SQL o PySpark

In [0]:
from pyspark.sql import (
    SparkSession,
    types,
    functions as F,
)

from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = (
    SparkSession
    .builder
    .appName('cleaning_orders_dataset_with_pyspark')
    .getOrCreate()
)

schema = StructType([
    StructField("STAGE", StringType(), True),
    StructField("DATE", StringType(), True), 
    StructField("PENS", StringType(), True),
    StructField("PENS_HOME_SCORE", StringType(), True),
    StructField("PENS_AWAY_SCORE", StringType(), True),
    StructField("TEAM_NAME_HOME", StringType(), True),
    StructField("TEAM_NAME_AWAY", StringType(), True),
    StructField("TEAM_HOME_SCORE", IntegerType(), True),
    StructField("TEAM_AWAY_SCORE", IntegerType(), True),
    StructField("POSSESSION_HOME", StringType(), True),
    StructField("POSSESSION_AWAY", StringType(), True),
    StructField("TOTAL_SHOTS_HOME", IntegerType(), True),
    StructField("TOTAL_SHOTS_AWAY", IntegerType(), True),
    StructField("SHOTS_ON_TARGET_HOME", FloatType(), True),
    StructField("SHOTS_ON_TARGET_AWAY", FloatType(), True),
    StructField("DUELS_WON_HOME", StringType(), True),
    StructField("DUELS_WON_AWAY", StringType(), True),
    StructField("PREDICTION_TEAM_HOME_WIN", StringType(), True),
    StructField("PREDICTION_DRAW", StringType(), True),
    StructField("PREDICTION_TEAM_AWAY_WIN", StringType(), True),
    StructField("LOCATION", StringType(), True)
])

In [0]:
def clean():
    # Añade una columna con índice
    daf_with_index = daf.withColumn("row_id", monotonically_increasing_id())

    window_spec = Window.orderBy("row_id")

    # Obtiene el registro anterior con lag()
    daf_with_previous = daf_with_index.withColumn(
        "prev_value", 
        lag(daf_with_index["value"]).over(window_spec)
    )

    # Combina los valores de aquella fila que empiece por " con la anterior
    daf_combined = daf_with_previous.withColumn(
        "final_value",
        when(substring(daf_with_previous.value, 0, 1) == '"',
        concat(daf_with_previous.prev_value, daf_with_previous.value)
        ).otherwise(
        daf_with_previous.value
        )
    )

In [0]:
from pyspark.sql.window import Window

daf = spark.read.text("dbfs:/FileStore/Examen/uefa_2020.csv")

clean()

df = daf_combined.drop("value","row_id","prev_value")


daf = spark.read.text("dbfs:/FileStore/Examen/uefa_2021.csv")

clean()

df2 = daf_combined.drop("value","row_id","prev_value")


daf = spark.read.text("dbfs:/FileStore/Examen/uefa_2022.csv")

clean()

df3 = daf_combined.drop("value","row_id","prev_value")


In [0]:
# Pasa a formato de porcentaje en decimal todas las columnas de porcentaje en los tres dataframes de los 3 csv

df = spark.read.csv("dbfs:/FileStore/Examen/uefa_2020.csv", header=True, schema=schema)

df = df.withColumn("DATE", trim(df.DATE))
df = df.withColumn("DATE", to_date(df.DATE, "d-M-y"))

df = df.withColumn("POSSESSION_HOME", regexp_replace(df.POSSESSION_HOME, '%', '')/100)
df = df.withColumn("POSSESSION_AWAY", regexp_replace(df.POSSESSION_AWAY, '%', '')/100)
df = df.withColumn("DUELS_WON_HOME", regexp_replace(df.DUELS_WON_HOME, '%', '')/100)
df = df.withColumn("DUELS_WON_AWAY", regexp_replace(df.DUELS_WON_AWAY, '%', '')/100)
df = df.withColumn("PREDICTION_TEAM_HOME_WIN", regexp_replace(df.PREDICTION_TEAM_HOME_WIN, '%', '')/100)
df = df.withColumn("PREDICTION_DRAW", regexp_replace(df.PREDICTION_DRAW, '%', '')/100)
df = df.withColumn("PREDICTION_TEAM_AWAY_WIN", regexp_replace(df.PREDICTION_TEAM_AWAY_WIN, '%', '')/100)



df2 = spark.read.csv("dbfs:/FileStore/Examen/uefa_2021.csv", header=True, schema=schema)

df2 = df2.withColumn("DATE", trim(df2.DATE))
df2 = df2.withColumn("DATE", to_date(df2.DATE, "d-M-y"))

df2 = df2.withColumn("POSSESSION_HOME", regexp_replace(df2.POSSESSION_HOME, '%', '')/100)
df2 = df2.withColumn("POSSESSION_AWAY", regexp_replace(df2.POSSESSION_AWAY, '%', '')/100)
df2 = df2.withColumn("DUELS_WON_HOME", regexp_replace(df2.DUELS_WON_HOME, '%', '')/100)
df2 = df2.withColumn("DUELS_WON_AWAY", regexp_replace(df2.DUELS_WON_AWAY, '%', '')/100)
df2 = df2.withColumn("PREDICTION_TEAM_HOME_WIN", regexp_replace(df2.PREDICTION_TEAM_HOME_WIN, '%', '')/100)
df2 = df2.withColumn("PREDICTION_DRAW", regexp_replace(df2.PREDICTION_DRAW, '%', '')/100)
df2 = df2.withColumn("PREDICTION_TEAM_AWAY_WIN", regexp_replace(df2.PREDICTION_TEAM_AWAY_WIN, '%', '')/100)



df3 = spark.read.csv("dbfs:/FileStore/Examen/uefa_2022.csv", header=True, schema=schema)

df3 = df3.withColumn("DATE", trim(df3.DATE))
df3 = df3.withColumn("DATE", to_date(df3.DATE, "d-M-y"))

df3 = df3.withColumn("POSSESSION_HOME", regexp_replace(df3.POSSESSION_HOME, '%', '')/100)
df3 = df3.withColumn("POSSESSION_AWAY", regexp_replace(df3.POSSESSION_AWAY, '%', '')/100)
df3 = df3.withColumn("DUELS_WON_HOME", regexp_replace(df3.DUELS_WON_HOME, '%', '')/100)
df3 = df3.withColumn("DUELS_WON_AWAY", regexp_replace(df3.DUELS_WON_AWAY, '%', '')/100)
df3 = df3.withColumn("PREDICTION_TEAM_HOME_WIN", regexp_replace(df3.PREDICTION_TEAM_HOME_WIN, '%', '')/100)
df3 = df3.withColumn("PREDICTION_DRAW", regexp_replace(df3.PREDICTION_DRAW, '%', '')/100)
df3 = df3.withColumn("PREDICTION_TEAM_AWAY_WIN", regexp_replace(df3.PREDICTION_TEAM_AWAY_WIN, '%', '')/100)



df = df.na.drop(subset=["DATE"])
df2 = df2.na.drop(subset=["DATE"])
df3 = df3.na.drop(subset=["DATE"])

df3.display()

STAGE,DATE,PENS,PENS_HOME_SCORE,PENS_AWAY_SCORE,TEAM_NAME_HOME,TEAM_NAME_AWAY,TEAM_HOME_SCORE,TEAM_AWAY_SCORE,POSSESSION_HOME,POSSESSION_AWAY,TOTAL_SHOTS_HOME,TOTAL_SHOTS_AWAY,SHOTS_ON_TARGET_HOME,SHOTS_ON_TARGET_AWAY,DUELS_WON_HOME,DUELS_WON_AWAY,PREDICTION_TEAM_HOME_WIN,PREDICTION_DRAW,PREDICTION_TEAM_AWAY_WIN,LOCATION
Group stage: Matchday 1,2022-09-06,FALSE,FALSE,FALSE,Dinamo Zagreb,Chelsea,1,0,0.31,0.69,6,15,3.0,3.0,0.44,0.56,0.49,0.1,0.51,Maksimir
Group stage: Matchday 1,2022-09-06,FALSE,FALSE,FALSE,Salzburg,AC Milan,1,1,0.34,0.66,16,14,4.0,3.0,0.38,0.62,0.41,0.09,0.59,Red Bull Arena
Group stage: Matchday 1,2022-09-06,FALSE,FALSE,FALSE,Celtic,Real Madrid,0,3,0.35,0.65,10,12,4.0,6.0,0.46,0.54,0.47,0.08,0.53,Celtic Park
Group stage: Matchday 1,2022-09-06,FALSE,FALSE,FALSE,RB Leipzig,Shakhtar Donetsk,1,4,0.68,0.32,13,5,2.0,4.0,0.47,0.53,0.49,0.08,0.51,Red Bull Arena
Group stage: Matchday 1,2022-09-06,FALSE,FALSE,FALSE,Borussia Dortmund,K�benhavn,3,0,0.63,0.37,15,7,4.0,2.0,0.49,0.51,0.45,0.06,0.55,Signal Iduna Park
Group stage: Matchday 1,2022-09-06,FALSE,FALSE,FALSE,Sevilla,Manchester City,0,4,0.38,0.62,6,24,1.0,10.0,0.42,0.58,0.43,0.02,0.57,Ram�n S�nchez Pizju�n
Group stage: Matchday 1,2022-09-06,FALSE,FALSE,FALSE,Paris Saint-Germain,Juventus,2,1,0.58,0.42,15,13,6.0,4.0,0.36,0.64,0.45,0.03,0.55,Parc des Princes
Group stage: Matchday 1,2022-09-06,FALSE,FALSE,FALSE,Benfica,Maccabi Haifa,2,0,0.57,0.43,14,7,5.0,2.0,0.46,0.54,0.42,0.08,0.58,Stadion da Luz
Group stage: Matchday 1,2022-09-07,FALSE,FALSE,FALSE,Ajax,Rangers,4,0,0.76,0.24,17,4,9.0,1.0,0.47,0.53,0.45,0.03,0.55,Amsterdam Arena
Group stage: Matchday 1,2022-09-07,FALSE,FALSE,FALSE,Napoli,Liverpool,4,1,0.38,0.62,18,15,9.0,7.0,0.44,0.56,0.45,0.08,0.55,San Paolo


1. Encuentra los 3 equipos que anotaron más goles jugando en su estadio en la UEFA Champions League 2020-21. El resultado debe contener dos columnas: TEAM_NAME_HOME y TEAM_HOME_SCORE ordenadas en orden descendente de TEAM_HOME_SCORE. Guarda la consulta (o el notebook) como TEAM_HOME_WITH_MOST_GOALS.

In [0]:
dataframe_1 = df.select("TEAM_NAME_HOME", "TEAM_HOME_SCORE").sort("TEAM_HOME_SCORE", ascending=False).limit(3)
dataframe_1.display()

TEAM_NAME_HOME,TEAM_HOME_SCORE
PSG,5
Manchester United,5
Barcelona,5


2. Encuentra el equipo con posesión mayoritaria la mayor cantidad de veces durante la UEFA Champions League 2021-22. El resultado debe incluir dos columnas: TEAM_NAME y GAME_COUNT, que es la cantidad de veces que el equipo tuvo posesión mayoritaria durante un partido de fútbol. Guarda esta consulta (o el notebook) como TEAM_WITH_MAJORITY_POSSESSION

In [0]:
# Toma el valor máximo de posesiones tanto para el equipo local como para el visitante de la jornada 2021-2022 (Supongo que cada temporada empieza el 1 de enero (no sé de fútbol))
dataframe_2_1 = df2.sort("POSSESSION_HOME", ascending=False).limit(1)
dataframe_2_2 = df2.sort("POSSESSION_AWAY", ascending=False).limit(1)

# Compara el valor más alto de posesión del equipo local contra las del equipo visitante y se queda la más alta, mostrando los datos en dos columnas
if dataframe_2_1.select("POSSESSION_HOME").collect()[0][0] > dataframe_2_2.select("POSSESSION_AWAY").collect()[0][0]:
    dataframe_2 = dataframe_2_1
    dataframe_2 = dataframe_2.select(dataframe_2.TEAM_NAME_HOME.alias("TEAM_NAME"), dataframe_2.POSSESSION_HOME.alias("GAME_COUNT"))
else:
    dataframe_2 = dataframe_2_2
    dataframe_2 = dataframe_2.select(dataframe_2.TEAM_NAME_AWAY.alias("TEAM_NAME"), dataframe_2.POSSESSION_AWAY.alias("GAME_COUNT"))



dataframe_2.display()

TEAM_NAME,GAME_COUNT
Liverpool,0.74


3. Encuentra la lista de equipos de cada fase del juego que ganaron el duelo en un partido pero terminaron perdiendo el juego en el Campeonato de la UEFA 2022-23. El resultado debe contener dos columnas: STAGE y TEAM_LOST. Guarda la consulta (o el notebook) como TEAM_WON_DUEL_LOST_GAME_STAGE_WISE.

In [0]:
dataframe_3 = df3.filter(
                        ((
                            (df3.DUELS_WON_HOME > df3.DUELS_WON_AWAY) &
                            (df3.TEAM_HOME_SCORE < df3.TEAM_AWAY_SCORE)
                        ) | 
                        (
                            (df3.DUELS_WON_HOME < df3.DUELS_WON_AWAY) & 
                            (df3.TEAM_HOME_SCORE > df3.TEAM_AWAY_SCORE)
                        ))).withColumn("TEAM_LOST", when(
                            df3.TEAM_HOME_SCORE > df3.TEAM_AWAY_SCORE,
                            df3.TEAM_NAME_AWAY
                        ).otherwise(df3.TEAM_NAME_HOME)).select("STAGE", "TEAM_LOST")

dataframe_3.display()

STAGE,TEAM_LOST
Group stage: Matchday 1,Chelsea
Group stage: Matchday 1,K�benhavn
Group stage: Matchday 1,Juventus
Group stage: Matchday 1,Maccabi Haifa
Group stage: Matchday 1,Rangers
Group stage: Matchday 1,Liverpool
Group stage: Matchday 1,Porto
Group stage: Matchday 1,Bayer Leverkusen
Group stage: Matchday 1,Viktoria Plze?
Group stage: Matchday 1,Marseille
